In [2]:
os.chdir('..')

In [50]:
import google

In [3]:
import os
import numpy as np
import pandas as pd
from google.cloud import bigquery
from smallscout.params import *
import db_dtypes

In [4]:
df = pd.read_csv('raw_data/data_for_preprocessing.csv', index_col=0)
df.shape

(170124, 59)

In [18]:
df.head()

,TICKER,date,quarter,Assets,AssetsCurrent,Cash,AssetsNoncurrent,Liabilities,LiabilitiesCurrent,LiabilitiesNoncurrent,...,EffectOfExchangeRateFinal,CashPeriodIncreaseDecreaseIncludingExRateEffectFinal,afs,sic_2d,GDP,interest_rate,unemployment_rate,median_cpi,small_cap,micro_cap
0,AIR,2011-02-28,2011-Q1,1.655991e+09,9.278390e+08,54716000.0,409295000.0,8.513950e+08,419182000.0,432213000.0,...,24000.0,-24654000.0,1-LAF,37,-1.361790,0.16,8.9,2.816969,1,0
1,AIR,2011-05-31,2011-Q2,1.703727e+09,9.139850e+08,57433000.0,465365000.0,8.684380e+08,416010000.0,452428000.0,...,27000.0,-21937000.0,1-LAF,37,2.455302,0.09,9.1,2.305816,1,0
2,AIR,2011-08-31,2011-Q3,1.752372e+09,9.442470e+08,35523000.0,472856000.0,9.032430e+08,350085000.0,553158000.0,...,-96000.0,-21910000.0,1-LAF,37,-0.043356,0.10,9.1,3.177971,1,0
3,AIR,2011-11-30,2011-Q4,1.821612e+09,9.550530e+08,27870000.0,521431000.0,9.582200e+08,374944000.0,583276000.0,...,477000.0,-29563000.0,1-LAF,37,5.124817,0.08,8.6,1.847014,1,0
4,AIR,2012-02-29,2012-Q1,2.220293e+09,1.065389e+09,59294000.0,797765000.0,1.328974e+09,560986000.0,767988000.0,...,851000.0,1861000.0,1-LAF,37,2.365306,0.10,8.3,1.684552,1,0


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 170124 entries, 0 to 170123
Data columns (total 49 columns):
 #   Column                                                           Non-Null Count   Dtype  
---  ------                                                           --------------   -----  
 0   TICKER                                                           170124 non-null  object 
 1   date                                                             170124 non-null  object 
 2   quarter                                                          170124 non-null  object 
 3   Assets                                                           170124 non-null  float64
 4   AssetsCurrent                                                    170124 non-null  float64
 5   Cash                                                             170124 non-null  float64
 6   AssetsNoncurrent                                                 170124 non-null  float64
 7   Liabilities                       

In [6]:
df.columns

Index(['cik', 'date', 'Assets', 'AssetsCurrent', 'Cash', 'AssetsNoncurrent',
       'Liabilities', 'LiabilitiesCurrent', 'LiabilitiesNoncurrent', 'Equity',
       'HolderEquity', 'RetainedEarnings', 'AdditionalPaidInCapital',
       'TreasuryStockValue', 'TemporaryEquity', 'RedeemableEquity',
       'LiabilitiesAndEquity', 'Revenues', 'CostOfRevenue', 'GrossProfit',
       'OperatingExpenses', 'OperatingIncomeLoss',
       'IncomeLossFromContinuingOperationsBeforeIncomeTaxExpenseBenefit',
       'AllIncomeTaxExpenseBenefit', 'IncomeLossFromContinuingOperations',
       'IncomeLossFromDiscontinuedOperationsNetOfTax', 'ProfitLoss',
       'NetIncomeLossAttributableToNoncontrollingInterest', 'NetIncomeLoss',
       'NetCashProvidedByUsedInOperatingActivitiesContinuingOperations',
       'NetCashProvidedByUsedInFinancingActivitiesContinuingOperations',
       'NetCashProvidedByUsedInInvestingActivitiesContinuingOperations',
       'NetCashProvidedByUsedInOperatingActivities',
       'NetCa

In [7]:
cols = ['TICKER', 'date', 'quarter', 'Assets', 'AssetsCurrent', 'Cash', 'AssetsNoncurrent', 'Liabilities', 'LiabilitiesCurrent', 
        'LiabilitiesNoncurrent', 'Equity', 'HolderEquity', 'RetainedEarnings', 'AdditionalPaidInCapital', 
        'TreasuryStockValue', 'TemporaryEquity', 'RedeemableEquity', 'LiabilitiesAndEquity', 'Revenues', 
        'CostOfRevenue', 'GrossProfit', 'OperatingExpenses', 'OperatingIncomeLoss', 
        'IncomeLossFromContinuingOperationsBeforeIncomeTaxExpenseBenefit', 'AllIncomeTaxExpenseBenefit', 
        'IncomeLossFromContinuingOperations', 'IncomeLossFromDiscontinuedOperationsNetOfTax', 'ProfitLoss', 
        'NetIncomeLossAttributableToNoncontrollingInterest', 'NetIncomeLoss', 
        'NetCashProvidedByUsedInOperatingActivitiesContinuingOperations', 
        'NetCashProvidedByUsedInFinancingActivitiesContinuingOperations', 
        'NetCashProvidedByUsedInInvestingActivitiesContinuingOperations', 
        'NetCashProvidedByUsedInOperatingActivities', 'NetCashProvidedByUsedInFinancingActivities', 
        'NetCashProvidedByUsedInInvestingActivities',
       'CashProvidedByUsedInOperatingActivitiesDiscontinuedOperations', 
        'CashProvidedByUsedInInvestingActivitiesDiscontinuedOperations', 
        'CashProvidedByUsedInFinancingActivitiesDiscontinuedOperations', 'EffectOfExchangeRateFinal', 
        'CashPeriodIncreaseDecreaseIncludingExRateEffectFinal', 'afs', 'sic_2d', 'GDP', 'interest_rate', 
        'unemployment_rate', 'median_cpi', 'small_cap', 'micro_cap']

In [8]:
df = df[cols]

In [9]:
df.columns

Index(['TICKER', 'date', 'quarter', 'Assets', 'AssetsCurrent', 'Cash',
       'AssetsNoncurrent', 'Liabilities', 'LiabilitiesCurrent',
       'LiabilitiesNoncurrent', 'Equity', 'HolderEquity', 'RetainedEarnings',
       'AdditionalPaidInCapital', 'TreasuryStockValue', 'TemporaryEquity',
       'RedeemableEquity', 'LiabilitiesAndEquity', 'Revenues', 'CostOfRevenue',
       'GrossProfit', 'OperatingExpenses', 'OperatingIncomeLoss',
       'IncomeLossFromContinuingOperationsBeforeIncomeTaxExpenseBenefit',
       'AllIncomeTaxExpenseBenefit', 'IncomeLossFromContinuingOperations',
       'IncomeLossFromDiscontinuedOperationsNetOfTax', 'ProfitLoss',
       'NetIncomeLossAttributableToNoncontrollingInterest', 'NetIncomeLoss',
       'NetCashProvidedByUsedInOperatingActivitiesContinuingOperations',
       'NetCashProvidedByUsedInFinancingActivitiesContinuingOperations',
       'NetCashProvidedByUsedInInvestingActivitiesContinuingOperations',
       'NetCashProvidedByUsedInOperatingActivities',

In [10]:
def load_data_to_bq(
        data: pd.DataFrame,
        gcp_project:str,
        bq_dataset:str,
        table: str,
        truncate: bool
    ) -> None:
    """
    - Save the DataFrame to BigQuery
    - Empty the table beforehand if `truncate` is True, append otherwise
    """

    assert isinstance(data, pd.DataFrame)
    full_table_name = f"{gcp_project}.{bq_dataset}.{table}"
    #data.columns = ['pickup_datetime'] + [f"_{i}" for i in range(1,len(data.columns)-1)] + ['fare_amount']

    # Load data onto full_table_name
    client = bigquery.Client(gcp_project)
    write_mode = "WRITE_TRUNCATE" if truncate else "WRITE_APPEND"
    job_config = bigquery.LoadJobConfig(write_disposition=write_mode)

    job = client.load_table_from_dataframe(data, full_table_name, job_config=job_config)
    result = job.result()

    # 🎯 HINT for "*** TypeError: expected bytes, int found":
    # After preprocessing the data, your original column names are gone (print it to check),
    # so ensure that your column names are *strings* that start with either
    # a *letter* or an *underscore*, as BQ does not accept anything else


    print(f"✅ Data saved to bigquery, with shape {data.shape}")



In [11]:
full_table_name = f"{GCP_PROJECT}.{BQ_DATASET}.{BQ_REGION}"
full_table_name

'lw-bootcamp-429514.smallcapscout.EU'

In [84]:
load_data_to_bq(data=df, gcp_project=GCP_PROJECT, bq_dataset=BQ_DATASET, table=BQ_REGION, truncate=False)


✅ Data saved to bigquery, with shape (170124, 49)


In [13]:
ticker = 'AAPL'
quarter = df.quarter.iloc[-1]
quarter

'2023-Q4'

In [14]:
query = f"""       
    SELECT *
    FROM {GCP_PROJECT}.{BQ_DATASET}.{BQ_REGION}
    WHERE TICKER = '{ticker}'
    ORDER BY DATE
    """

client = bigquery.Client(project=GCP_PROJECT)
query_job = client.query(query)
result = query_job.result()
data = result.to_dataframe()

/Users/christophernance/.pyenv/versions/3.10.6/envs/small-cap-scout/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [16]:
data.dtypes

TICKER                                                              object
date                                                                object
quarter                                                             object
Assets                                                             float64
AssetsCurrent                                                      float64
Cash                                                               float64
AssetsNoncurrent                                                   float64
Liabilities                                                        float64
LiabilitiesCurrent                                                 float64
LiabilitiesNoncurrent                                              float64
Equity                                                             float64
HolderEquity                                                       float64
RetainedEarnings                                                   float64
AdditionalPaidInCapital  

In [15]:
data

,TICKER,date,quarter,Assets,AssetsCurrent,Cash,AssetsNoncurrent,Liabilities,LiabilitiesCurrent,LiabilitiesNoncurrent,...,EffectOfExchangeRateFinal,CashPeriodIncreaseDecreaseIncludingExRateEffectFinal,afs,sic_2d,GDP,interest_rate,unemployment_rate,median_cpi,small_cap,micro_cap
0,AAPL,2010-03-31,2010-Q1,5.705700e+10,3.233600e+10,1.001800e+10,2.472100e+10,1.770900e+10,1.222900e+10,5.480000e+09,...,0.0,4.755000e+09,1-LAF,35,4.165865,0.16,9.7,-0.286064,0,0
1,AAPL,2010-06-30,2010-Q2,6.472500e+10,3.603300e+10,9.705000e+09,2.869200e+10,2.161400e+10,1.561200e+10,6.002000e+09,...,0.0,4.442000e+09,1-LAF,35,3.114905,0.18,9.5,1.264753,0,0
2,AAPL,2010-09-30,2010-Q3,7.518300e+10,4.167800e+10,1.126100e+10,3.350500e+10,2.739200e+10,2.072200e+10,6.670000e+09,...,0.0,5.998000e+09,1-LAF,35,3.121818,0.19,9.6,1.536272,0,0
3,AAPL,2010-12-31,2010-Q4,8.674200e+10,4.392700e+10,1.073400e+10,4.281500e+10,3.207600e+10,2.379500e+10,8.281000e+09,...,0.0,-5.270000e+08,1-LAF,35,-1.173723,0.18,9.4,1.393380,0,0
4,AAPL,2011-03-31,2011-Q1,9.490400e+10,4.699700e+10,1.597800e+10,4.790700e+10,3.342700e+10,2.432700e+10,9.100000e+09,...,0.0,4.717000e+09,1-LAF,35,4.397708,0.14,8.8,1.744190,0,0
5,AAPL,2011-06-30,2011-Q2,1.067580e+11,4.689800e+10,1.209100e+10,5.986000e+10,3.741500e+10,2.685900e+10,1.055600e+10,...,0.0,8.300000e+08,1-LAF,35,-0.381671,0.09,9.2,1.926728,0,0
6,AAPL,2011-09-30,2011-Q3,1.163710e+11,4.498800e+10,9.815000e+09,7.138300e+10,3.975600e+10,2.797000e+10,1.178600e+10,...,0.0,-1.446000e+09,1-LAF,35,3.270582,0.08,9.1,2.491977,0,0
7,AAPL,2011-12-31,2011-Q4,1.386810e+11,5.477100e+10,1.031000e+10,8.391000e+10,4.862700e+10,3.460700e+10,1.402000e+10,...,0.0,4.950000e+08,1-LAF,35,4.413609,0.07,8.5,2.374333,0,0
8,AAPL,2012-03-31,2012-Q1,1.509340e+11,5.071200e+10,1.012100e+10,1.002220e+11,4.843600e+10,3.203600e+10,1.640000e+10,...,0.0,3.060000e+08,1-LAF,35,1.765497,0.13,8.2,2.420296,0,0
9,AAPL,2012-06-30,2012-Q2,1.628960e+11,5.194300e+10,7.945000e+09,1.109530e+11,5.115000e+10,3.306000e+10,1.809000e+10,...,0.0,-1.870000e+09,1-LAF,35,0.920124,0.16,8.2,1.922063,0,0


In [47]:
data.iloc[data.index[data.quarter==quarter]]

,TICKER,date,quarter,Assets,AssetsCurrent,Cash,AssetsNoncurrent,Liabilities,LiabilitiesCurrent,LiabilitiesNoncurrent,...,EffectOfExchangeRateFinal,CashPeriodIncreaseDecreaseIncludingExRateEffectFinal,afs,sic_2d,GDP,interest_rate,unemployment_rate,median_cpi,small_cap,micro_cap
55,AAPL,2023-12-31,2023-Q4,3.535140e+11,1.436920e+11,4.076000e+10,2.098220e+11,2.794140e+11,1.339730e+11,1.454410e+11,...,0.0,1.123700e+10,None,na,1.388811,5.33,3.7,4.160125,0,0


In [49]:
horizon = 'quarter-ahead'
if horizon not in ['quarter-ahead', 'year-ahead', 'two-years-ahead']:
    raise ValueError (f"Unsupported horizon: {horizon}, must be quarter-ahead, year-ahead or two-years-ahead")